In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [5]:
!wget $data -O data-week-3.csv

--2025-10-15 12:13:44--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: 'data-week-3.csv’

data-week-3.csv     100%[===================>]  78,98K  --.-KB/s    in 0,08s   

2025-10-15 12:13:45 (1,02 MB/s) - 'data-week-3.csv’ saved [80876/80876]



In [8]:
df = pd.read_csv('data-week-3.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [9]:
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [10]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [15]:
# Имена колонок уже ок, но на всякий:
df.columns = df.columns.str.lower().str.replace(' ', '_')

cat_cols = ['lead_source', 'industry', 'employment_status', 'location']
num_cols = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
target = 'converted'

# Категориальные: заполнить пропуски 'NA' и убедиться в нормализации
for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].astype('string').fillna('NA').str.strip().str.lower().str.replace(' ', '_')

# Числовые: если вдруг пришли строками — привести; пропуски -> 0.0
for c in num_cols:
    if c in df.columns:
        if not np.issubdtype(df[c].dtype, np.number):
            df[c] = pd.to_numeric(df[c], errors='coerce')
        df[c] = df[c].fillna(0.0)

# Цель — оставить int (на случай неожиданных форматов)
if target in df.columns and df[target].dtype != 'int64':
    if df[target].dtype == 'object' or str(df[target].dtype).startswith('string'):
        df[target] = (df[target].str.strip().str.lower() == 'yes').astype(int)
    else:
        df[target] = df[target].astype(int)

In [16]:
df.head().T


,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,na,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,na,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [26]:
from sklearn.model_selection import train_test_split

# сначала test 20%
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

# затем из train_full делаем train/val ~ 53.6% / 26.4%
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)

# цель
y_train = df_train.converted.values
y_val   = df_val.converted.values

# признаки (без цели)
del df_train['converted']
del df_val['converted']


('lead_source', np.float64(0.03))

In [18]:
df_train_full.isnull().sum()



lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [20]:
df_train_full.converted.value_counts()

converted
1    715
0    454
Name: count, dtype: int64

In [21]:
global_mean = df_train_full.converted.mean()
round(global_mean, 3)



np.float64(0.612)

In [22]:
q1_answer = df['industry'].mode()[0]
q1_answer

'retail'

In [23]:
num = ['number_of_courses_viewed','annual_income','interaction_count','lead_score']
corr = df[num].corr()

pairs = [
    ('interaction_count','lead_score'),
    ('number_of_courses_viewed','lead_score'),
    ('number_of_courses_viewed','interaction_count'),
    ('annual_income','interaction_count'),
]

res = {p: corr.loc[p[0], p[1]] for p in pairs}
res, max(res, key=lambda k: abs(res[k]))


({('interaction_count', 'lead_score'): np.float64(0.009888182496913131),
  ('number_of_courses_viewed',
   'lead_score'): np.float64(-0.004878998354681276),
  ('number_of_courses_viewed',
   'interaction_count'): np.float64(-0.023565222882888037),
  ('annual_income', 'interaction_count'): np.float64(0.02703647240481443)},
 ('annual_income', 'interaction_count'))

In [24]:
global_conv = df_train_full.converted.mean()

for col in ['lead_source', 'industry', 'employment_status', 'location']:
    g = df_train_full.groupby(col).converted.agg(['count','mean']).rename(columns={'mean':'conv_rate'})
    g['diff'] = g['conv_rate'] - global_conv
    g['risk'] = g['conv_rate'] / global_conv
    print(f'\n=== {col} ===')
    display(g.sort_values('risk', ascending=False).head(10))



=== lead_source ===


,count,conv_rate,diff,risk
lead_source,,,,
referral,206,0.791262,0.179628,1.293686
na,97,0.639175,0.027541,1.045029
organic_search,230,0.626087,0.014453,1.023630
events,192,0.604167,-0.007467,0.987791
social_media,230,0.591304,-0.020330,0.966762
paid_ads,214,0.439252,-0.172382,0.718162



=== industry ===


,count,conv_rate,diff,risk
industry,,,,
education,157,0.751592,0.139958,1.228827
manufacturing,151,0.642384,0.030750,1.050276
other,156,0.602564,-0.009070,0.985171
healthcare,144,0.597222,-0.014412,0.976437
finance,155,0.587097,-0.024537,0.959883
retail,160,0.581250,-0.030384,0.950323
na,113,0.557522,-0.054112,0.911529
technology,133,0.548872,-0.062762,0.897387



=== employment_status ===


,count,conv_rate,diff,risk
employment_status,,,,
employed,273,0.666667,0.055033,1.089977
student,288,0.659722,0.048088,1.078623
self_employed,275,0.654545,0.042912,1.070159
na,73,0.534247,-0.077387,0.873474
unemployed,260,0.476923,-0.134711,0.779753



=== location ===


,count,conv_rate,diff,risk
location,,,,
na,50,0.660000,0.048366,1.079077
europe,176,0.636364,0.024730,1.040432
middle_east,157,0.630573,0.018939,1.030965
africa,149,0.617450,0.005816,1.009509
asia,151,0.615894,0.004260,1.006965
south_america,152,0.611842,0.000208,1.000340
north_america,186,0.580645,-0.030989,0.949335
australia,148,0.574324,-0.037310,0.939000


In [27]:
from sklearn.metrics import mutual_info_score
import pandas as pd

cat = ['lead_source', 'industry', 'employment_status', 'location']

df_mi = df_train[cat].apply(lambda s: mutual_info_score(s, y_train))
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')

df_mi_rounded = df_mi.round(2)

display(df_mi_rounded)


{'lead_source': np.float64(0.03),
 'industry': np.float64(0.01),
 'employment_status': np.float64(0.02),
 'location': np.float64(0.0)}

In [28]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical   = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']


In [29]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 1) словари признаков
train_dict = df_train[categorical + numerical].to_dict(orient='records')
val_dict   = df_val[categorical + numerical].to_dict(orient='records')

# 2) one-hot
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dict)
X_val   = dv.transform(val_dict)

# 3) логистическая регрессия (параметры из ДЗ)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# 4) accuracy на валидации (порог 0.5)
val_pred = (model.predict_proba(X_val)[:, 1] >= 0.5).astype(int)
acc = accuracy_score(y_val, val_pred)
print("Q4 validation accuracy:", round(acc, 2))


Q4 validation accuracy: 0.74


In [31]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical   = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

def train_acc(df_train, y_train, df_val, y_val, cats, nums):
    dv = DictVectorizer(sparse=True)
    Xtr = dv.fit_transform(df_train[cats + nums].to_dict(orient='records'))
    Xva = dv.transform(df_val[cats + nums].to_dict(orient='records'))
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(Xtr, y_train)
    pred = (model.predict_proba(Xva)[:, 1] >= 0.5).astype(int)
    return accuracy_score(y_val, pred)

# 1) База (со всеми фичами) — без округления
base_acc = train_acc(df_train, y_train, df_val, y_val, categorical, numerical)
print("Base val accuracy:", base_acc)

# 2) По очереди убираем каждую из кандидатных фич
candidates = ['industry', 'employment_status', 'lead_score']
diffs = {}
for f in candidates:
    cats = [c for c in categorical if c != f]
    nums = [n for n in numerical if n != f]
    acc_wo = train_acc(df_train, y_train, df_val, y_val, cats, nums)
    diffs[f] = base_acc - acc_wo

print("Diffs (base - without):", diffs)
# фича с наименьшей разницей
q5_feature = min(diffs, key=diffs.get)
print("Q5: feature with smallest diff ->", q5_feature, diffs[q5_feature])


Base val accuracy: 0.7435233160621761
Diffs (base - without): {'industry': -0.002590673575129543, 'employment_status': 0.002590673575129543, 'lead_score': 0.002590673575129543}
Q5: feature with smallest diff -> industry -0.002590673575129543


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Cs = [0.01, 0.1, 1, 10, 100]
accs = {}

Xtr = dv.fit_transform(df_train[categorical + numerical].to_dict('records'))
Xva = dv.transform(df_val[categorical + numerical].to_dict('records'))

for C in Cs:
    m = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    m.fit(Xtr, y_train)
    pred = (m.predict_proba(Xva)[:,1] >= 0.5).astype(int)
    accs[C] = round(accuracy_score(y_val, pred), 3)

accs, min([C for C,a in accs.items() if a==max(accs.values())])


({0.01: 0.749, 0.1: 0.744, 1: 0.744, 10: 0.744, 100: 0.744}, 0.01)